# Accessing LPD data

The Large Pixel Detector (LPD) 1M is made of 16 modules which record data separately.
`extra_data` includes convenient interfaces to access this data together.

This example stands by itself, but if you need more generic access to the data,
please see other examples, including [Reading data to analyse in memory](xpd_examples.ipynb)
and [Reading data train by train](iterate_trains.ipynb).

The example uses some empty sample files which are generated by this cell:

In [1]:
!python3 -m extra_data.tests.make_examples

Written examples.


First, let's load a run containing LPD data:

In [2]:
from extra_data import RunDirectory, by_index

run = RunDirectory('fxe_example_run/')
# Using only the first three trains to keep this example light:
run = run.select_trains(by_index[:3])

run.instrument_sources

frozenset({'FXE_DET_LPD1M-1/DET/0CH0:xtdf',
           'FXE_DET_LPD1M-1/DET/10CH0:xtdf',
           'FXE_DET_LPD1M-1/DET/11CH0:xtdf',
           'FXE_DET_LPD1M-1/DET/12CH0:xtdf',
           'FXE_DET_LPD1M-1/DET/13CH0:xtdf',
           'FXE_DET_LPD1M-1/DET/14CH0:xtdf',
           'FXE_DET_LPD1M-1/DET/15CH0:xtdf',
           'FXE_DET_LPD1M-1/DET/1CH0:xtdf',
           'FXE_DET_LPD1M-1/DET/2CH0:xtdf',
           'FXE_DET_LPD1M-1/DET/3CH0:xtdf',
           'FXE_DET_LPD1M-1/DET/4CH0:xtdf',
           'FXE_DET_LPD1M-1/DET/5CH0:xtdf',
           'FXE_DET_LPD1M-1/DET/6CH0:xtdf',
           'FXE_DET_LPD1M-1/DET/7CH0:xtdf',
           'FXE_DET_LPD1M-1/DET/8CH0:xtdf',
           'FXE_DET_LPD1M-1/DET/9CH0:xtdf',
           'FXE_XAD_GEC/CAM/CAMERA:daqOutput',
           'FXE_XAD_GEC/CAM/CAMERA_NODATA:daqOutput',
           'SA1_XTD2_XGM/DOOCS/MAIN:output',
           'SPB_XTD9_XGM/DOOCS/MAIN:output'})

Normal access methods give us each module separately:

In [3]:
data_module0 = run['FXE_DET_LPD1M-1/DET/0CH0:xtdf', 'image.data'].ndarray()
data_module0.shape

(384, 1, 256, 256)

The class `extra_data.components.LPD1M` can piece these together:

In [4]:
from extra_data.components import LPD1M
lpd = LPD1M(run)
lpd

<LPD1M: Data interface for detector 'FXE_DET_LPD1M-1' - raw data with 16 modules>

In [5]:
image_data = lpd.get_array('image.data')
print("Data shape:", image_data.shape)
print("Dimensions:", image_data.dims)

Data shape: (16, 3, 128, 256, 256)
Dimensions: ('module', 'train', 'pulse', 'slow_scan', 'fast_scan')


**Note:** This class pulls the data together, but it doesn't know how the modules are physically arranged,
so it can't produce a detector image. Other examples show how to use detector geometry to produce images.

You can also select only certain modules of the detector. For example, modules 2 (Q1M3), 7 (Q2M4), 8 (Q3M1) and 13 (Q4M2) are the four modules around the center of the detector:

In [6]:
lpd = LPD1M(run, modules=[2, 7, 8, 13])
image_data = lpd.get_array('image.data')
print("Data shape:", image_data.shape)
print("Dimensions:", image_data.dims)

print()
print("Data for one pulse:")
print(image_data.sel(train=10000, pulse=0))

Data shape: (4, 3, 128, 256, 256)
Dimensions: ('module', 'train', 'pulse', 'slow_scan', 'fast_scan')

Data for one pulse:
<xarray.DataArray (module: 4, slow_scan: 256, fast_scan: 256)> Size: 524kB
array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, .

The returned array is an *xarray* object with labelled axes.
See [Indexing and selecting data](https://xarray.pydata.org/en/stable/indexing.html) in the xarray docs
for more on what you can do with it.

This interface also supports iterating train-by-train through detector data, giving labelled arrays again:

In [7]:
for tid, train_data in lpd.trains(pulses=by_index[:16]):
    print("Train", tid)
    print("Keys in data:", sorted(train_data.keys()))
    print("Image data shape:", train_data['image.data'].shape)
    print()

Train 10000
Keys in data: ['detector.data', 'detector.trainId', 'header.dataId', 'header.linkId', 'header.magicNumberBegin', 'header.majorTrainFormatVersion', 'header.minorTrainFormatVersion', 'header.pulseCount', 'header.reserved', 'header.trainId', 'image.cellId', 'image.data', 'image.length', 'image.pulseId', 'image.status', 'image.trainId', 'trailer.checksum', 'trailer.magicNumberEnd', 'trailer.status', 'trailer.trainId']
Image data shape: (4, 1, 16, 256, 256)

Train 10001
Keys in data: ['detector.data', 'detector.trainId', 'header.dataId', 'header.linkId', 'header.magicNumberBegin', 'header.majorTrainFormatVersion', 'header.minorTrainFormatVersion', 'header.pulseCount', 'header.reserved', 'header.trainId', 'image.cellId', 'image.data', 'image.length', 'image.pulseId', 'image.status', 'image.trainId', 'trailer.checksum', 'trailer.magicNumberEnd', 'trailer.status', 'trailer.trainId']
Image data shape: (4, 1, 16, 256, 256)

Train 10002
Keys in data: ['detector.data', 'detector.trainI

LPD data may also be recorded in *parallel gain* mode, resulting in high-, medium- and low-gain frames for each pulse. To read this kind of data with the correct labels, use `LPD1M(run, parallel_gain=True)`. This will retrieve data with an extra gain dimension, labelled with 0, 1 and 2 for high-, medium- and low-gain respectively.

## Reading calibrated data

Now let's take a look at some calibrated LPD data. This will work only at the EuXFEL Maxwell cluster since the data is store there. To read the data processed with the offline calibration pipeline we set 'data' argument in `open_run` to 'proc', and to keep the example small we will read data only for the first two trains:

In [8]:
import numpy as np
import xarray as xr
from extra_data import open_run, by_index
from extra_data.components import LPD1M

run_data = open_run(proposal=700002, run=117, data='proc')
run_data_sel = run_data.select_trains(by_index[:2])
lpd_proc = LPD1M(run_data_sel)
lpd_data = lpd_proc.get_array('image.data')

print("Data array shape:", lpd_data.shape)
print("Dimensions:", lpd_data.dims)
print()
print("Data from a [29:34, 126:131] slice of module 0 for one pulse:")
print(lpd_data[0, 0, 0, 29:34, 126:131])

Data array shape: (16, 2, 1, 256, 256)
Dimensions: ('module', 'train', 'pulse', 'slow_scan', 'fast_scan')

Data from a [29:34, 126:131] slice of module 0 for one pulse:
<xarray.DataArray (slow_scan: 5, fast_scan: 5)> Size: 100B
array([[  0.       ,   0.       ,   5.3925676,   0.       ,  11.018078 ],
       [  0.       ,   0.       ,   0.       ,  -7.874025 , -10.992689 ],
       [  0.       ,   0.       ,   0.       ,   0.       ,   4.4062967],
       [ 16.685514 ,  37.263496 ,   2.100265 ,  44.686913 ,  15.549667 ],
       [ 43.927513 ,   8.110336 ,  10.531646 ,  63.739063 ,  25.869213 ]],
      dtype=float32)
Coordinates:
    train    uint64 8B 1713878183
    pulse    uint64 8B 8
    module   int64 8B 0
Dimensions without coordinates: slow_scan, fast_scan


While working with the real detector data it is worth to remember that some detector pixels may be defective and provide wrong signal. Most of such abnormal pixels are identified by the offline calibration pipeline and stored as a dynamic (per detector image) mask. This mask represents good pixels with `0` and bad pixels are bitwise encoded according to the [enumeration of bad pixel reasons](https://extra.readthedocs.io/en/latest/calibration/#extra.calibration.BadPixels).

In [9]:
lpd_mask = lpd_proc.get_array('image.mask')
print("Mask from a [29:34, 126:131] slice of module 0 for one pulse:")
print(lpd_mask[0, 0, 0, 29:34, 126:131])

Mask from a [29:34, 126:131] slice of module 0 for one pulse:
<xarray.DataArray (slow_scan: 5, fast_scan: 5)> Size: 100B
array([[35, 35,  0,  0,  0],
       [35, 35, 32,  0,  0],
       [35, 35, 32, 32,  0],
       [ 0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0]], dtype=uint32)
Coordinates:
    train    uint64 8B 1713878183
    pulse    uint64 8B 8
    module   int64 8B 0
Dimensions without coordinates: slow_scan, fast_scan


Applying mask to data:

In [10]:
lpd_data_masked_01 = lpd_data.copy()
lpd_data_masked_01.values[(lpd_mask > 0).values] = np.nan
print("Masked data from a [29:34, 126:131] slice of module 0 for one pulse:")
print(lpd_data_masked_01[0, 0, 0, 29:34, 126:131])

Masked data from a [29:34, 126:131] slice of module 0 for one pulse:
<xarray.DataArray (slow_scan: 5, fast_scan: 5)> Size: 100B
array([[        nan,         nan,   5.3925676,   0.       ,  11.018078 ],
       [        nan,         nan,         nan,  -7.874025 , -10.992689 ],
       [        nan,         nan,         nan,         nan,   4.4062967],
       [ 16.685514 ,  37.263496 ,   2.100265 ,  44.686913 ,  15.549667 ],
       [ 43.927513 ,   8.110336 ,  10.531646 ,  63.739063 ,  25.869213 ]],
      dtype=float32)
Coordinates:
    train    uint64 8B 1713878183
    pulse    uint64 8B 8
    module   int64 8B 0
Dimensions without coordinates: slow_scan, fast_scan


The same can be conveniently achieved using the `.masked_data()` method that combines data and mask for you:

In [11]:
lpd_data_masked_02 = lpd_proc.masked_data().xarray()
print("Masked data from a [29:34, 126:131] slice of module 0 for one pulse:")
print(lpd_data_masked_02[0, 0, 29:34, 126:131])

Masked data from a [29:34, 126:131] slice of module 0 for one pulse:
<xarray.DataArray (slow_scan: 5, fast_scan: 5)> Size: 100B
array([[        nan,         nan,   5.3925676,   0.       ,  11.018078 ],
       [        nan,         nan,         nan,  -7.874025 , -10.992689 ],
       [        nan,         nan,         nan,         nan,   4.4062967],
       [ 16.685514 ,  37.263496 ,   2.100265 ,  44.686913 ,  15.549667 ],
       [ 43.927513 ,   8.110336 ,  10.531646 ,  63.739063 ,  25.869213 ]],
      dtype=float32)
Coordinates:
    train_pulse  object 8B (1713878183, 8)
    train        uint64 8B 1713878183
    pulse        uint64 8B 8
    module       int64 8B 0
Dimensions without coordinates: slow_scan, fast_scan
